# Senior Design Team 1725, Classifying AIS, MM Solution Training
This file is a jupyter notebook containing
the training results for the "multiple models"
solution for the AIS Classification project.

Intent: showcase a training method that can work
        with any dataset size for a specific class
        while also being flexible for training new models
        that can handle other invasive species.

Basically, rather than having one model that handles all species,
we attempt to have multiple models chained together to handle identification

##Training Run: 2
This is the second version of the file. This time, we try to accomplish a basic training with 200 images from the none and phragmites class

- We are using an A100 GPU instance

## Dataset Mounting:
First, we want to access our dataset. To do this, we mount our sharepoint/OneDrive to colab. Can be done by following [this](https://medium.com/@pratikmukherjee32/mount-onedrive-to-google-colab-f941a3a96a79) guide.

You must first run the rclone config *locally*, then you can get it on Colab. The reason for this is because we need an API key for the mount to work properly, which requires web browser access (not possible in Colab).

In [ ]:
!wget https://downloads.rclone.org/v1.69.0/rclone-v1.69.0-linux-amd64.deb
!apt install ./rclone-v1.69.0-linux-amd64.deb

--2025-02-06 21:06:53--  https://downloads.rclone.org/v1.69.0/rclone-v1.69.0-linux-amd64.deb
Resolving downloads.rclone.org (downloads.rclone.org)... 95.217.6.16, 2a01:4f9:c012:7154::1
Connecting to downloads.rclone.org (downloads.rclone.org)|95.217.6.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23665802 (23M) [application/vnd.debian.binary-package]
Saving to: ‘rclone-v1.69.0-linux-amd64.deb’

rclone-v1.69.0-linu 100%[===================>]  22.57M  14.0MB/s    in 1.6s    

2025-02-06 21:06:55 (14.0 MB/s) - ‘rclone-v1.69.0-linux-amd64.deb’ saved [23665802/23665802]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'rclone' instead of './rclone-v1.69.0-linux-amd64.deb'
The following NEW packages will be installed:
  rclone
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 0 B/23.7 MB of archives.
After this operation, 71.0 MB of additional disk space will be used.
Get

In [ ]:
# We will need to pass some arguments for this to work,
# so make sure we still have the api key
# (Note: you need to copy the ENTIRE struct, as the values are read as an array)

# Also, make sure to clear the output! We don't want to have
# loose api keys floating around the internet!
!rclone config

In [ ]:
C
# Create onedrive directory in colab

In [ ]:
# Install fusermount3 to ensure proper mounting
!sudo apt-get -y install fuse3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following package was automatically installed and is no longer required:
  libfuse2
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libfuse3-3
The following packages will be REMOVED:
  fuse
The following NEW packages will be installed:
  fuse3 libfuse3-3
0 upgraded, 2 newly installed, 1 to remove and 19 not upgraded.
Need to get 106 kB of archives.
After this operation, 264 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfuse3-3 amd64 3.10.5-1build1 [81.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 fuse3 amd64 3.10.5-1build1 [24.7 kB]
Fetched 106 kB in 0s (449 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 

In [ ]:
!nohup rclone --vfs-cache-mode full mount onedrive: /content/onedrive &
# After these commands, we should have a onedrive folder in the files section
# within colab with everything we need!

nohup: appending output to 'nohup.out'


## Dataset Setup:
What we want to do next is copy the files from the datasets we want to use for creating our first few models.

Basically, we are copying the files to the colab environment because we don't want to overwrite what's going on with OneDrive and because we want a "fresh" place to store our training results

##Word of warning:
Doing this will consume disk resources on Colab, make sure you have enough space with the images you copy over!

In [ ]:
# Import utilities to take random image sampling and copy to target directory
import os
import shutil
import random

# Create a new directory specifically to hold dataset
!sudo mkdir /content/SenDsgnTraining

In [ ]:
# Code assistance from Joey George
# Dataset path:
source_dir = r"/content/onedrive/Training Images/Training"
target_dir = r"/content/SenDsgnTraining"

# Define classes for the smaller datasets
# The options we have are:
# "narrowleaf_cattail", "none", "phragmites", "purple_loosestrife"
classes = ["none", "phragmites"]
splits = {"train": 70, "val": 15, "test": 15}  # Percentage split

In [ ]:
# Create target directory structure
for split in splits:
    for class_name in classes:
        os.makedirs(os.path.join(target_dir, split, class_name), exist_ok=True)

In [ ]:
# Process each class
num_images = 200 # number of images we want to use for each class
for class_name in classes:
    source_path = os.path.join(source_dir, class_name)
    images = [f for f in os.listdir(source_path) if f.endswith(".jpg")]

    # Randomly sample images
    selected_images = random.sample(images, min(num_images, len(images)))

    # Distribute images across train, val, test
    split_counts = {k: int(v * len(selected_images) / 100) for k, v in splits.items()}
    start_idx = 0
    for split, count in split_counts.items():
        split_path = os.path.join(target_dir, split, class_name)
        for i in range(count):
            img_name = f"{start_idx}_{class_name}.jpg"
            shutil.copy(os.path.join(source_path, selected_images[start_idx]), os.path.join(split_path, img_name))
            start_idx += 1

## Training
Now that we've gotten our dataset mounted and our splits taken care of, we can begin training.

The best thing about the classification model is that no .yaml file is necessary, as the folder structure is enough to ensure the training, validation, and testing splits are recognized for each class!

In [ ]:
# Install Ultralytics and dependencies
%pip install ultralytics  # install
from ultralytics import YOLO, checks
checks()  # checks

Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 43.2/235.7 GB disk)


In [ ]:
# Load a model
model = YOLO("yolo11m-cls.pt")  # Load a pretrained model

# Train the model
results = model.train(data=target_dir, epochs=25, imgsz=640) # batch=48 not used this time

100%|██████████| 22.4M/22.4M [00:00<00:00, 162MB/s]


Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=classify, mode=train, model=yolo11m-cls.pt, data=/content/SenDsgnTraining, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

100%|██████████| 5.35M/5.35M [00:00<00:00, 59.2MB/s]


AMP: checks passed ✅


train: Scanning /content/SenDsgnTraining/train... 280 images, 0 corrupt: 100%|██████████| 280/280 [00:00<00:00, 1599.07it/s]

...

      25/25      6.32G    0.00193          8        640: 100%|██████████| 18/18 [00:14<00:00,  1.21it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00, 23.16it/s]

                   all          1          1



25 epochs completed in 0.136 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 20.9MB
Optimizer stripped from runs/classify/train/weights/best.pt, 20.9MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11m-cls summary (fused): 138 layers, 10,344,194 parameters, 0 gradients, 39.3 GFLOPs
train: /content/SenDsgnTraining/train... found 280 images in 2 classes ✅ 
val: /content/SenDsgnTraining/val... found 60 images in 2 classes ✅ 
test: /content/SenDsgnTraining/test... found 60 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00, 12.10it/s]


                   all          1          1
Speed: 0.4ms preprocess, 1.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


In [ ]:
# Just in case, we want to download the "runs" folder and save our training results
!zip -r /content/run2.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/classify/ (stored 0%)
  adding: content/runs/classify/train/ (stored 0%)
  adding: content/runs/classify/train/train_batch2.jpg (deflated 1%)
  adding: content/runs/classify/train/val_batch0_pred.jpg (deflated 0%)
  adding: content/runs/classify/train/train_batch1.jpg (deflated 1%)
  adding: content/runs/classify/train/results.png (deflated 11%)
  adding: content/runs/classify/train/events.out.tfevents.1738876534.19d4f559969d.1354.0 (deflated 93%)
  adding: content/runs/classify/train/confusion_matrix.png (deflated 36%)
  adding: content/runs/classify/train/args.yaml (deflated 52%)
  adding: content/runs/classify/train/weights/ (stored 0%)
  adding: content/runs/classify/train/weights/best.pt (deflated 8%)
  adding: content/runs/classify/train/weights/last.pt (deflated 8%)
  adding: content/runs/classify/train/val_batch0_labels.jpg (deflated 0%)
  adding: content/runs/classify/train/train_batch271.jpg (deflated 1%)
  adding: co